# Agent Library

In [ ]:
#| export
import inspect, re, json
from litellm import completion, ModelResponse

In [ ]:
#| export
type_map = {int: "integer", str: "string", float: "number", bool: "boolean"}

In [ ]:
#| export
def mk_tool_def(fn):
    s = inspect.getsource(fn).split(')')[0]
    matches = re.findall(r'(\w+):.+#\s*(.+?)\s*$', s, flags=re.MULTILINE)
    params = inspect.signature(fn).parameters  # Changed from add_numbers to fn
    return {
        "type": "function", 
        "function": {
            "name": fn.__name__, 
            "description": fn.__doc__,  
            "parameters": {
                "type": "object",
                "properties": {v: {'type': type_map[fn.__annotations__[v]], 'description': d} for v,d in matches},
                "required": [p for p in params.keys() if params[p].default == inspect.Parameter.empty]
            }
        }
    }

In [ ]:
#| export
def mk_msg(m, role='user'): return {"role":role, "content":m}

In [ ]:
#| export
def mk_tool_res(m, tc_id): return mk_msg(m, role='tool') | {"tool_call_id": tc_id}

In [ ]:
#| export
def ex_tool(tc):
    """Execute tool call"""
    fn = tc['function']
    res = str(globals()[fn['name']](**json.loads(fn['arguments'])))
    return mk_tool_res(res, tc['id'])

In [ ]:
#| export
def _repr_markdown_(self):
    tool_info = ''
    if self.choices[0].finish_reason == 'tool_calls':
        tc = self.choices[0].message.tool_calls[0]
        fn = tc.function
        tool_info = f'\n\n\tFunction call: `{fn.name}(**{fn.arguments})`'
    return (self.choices[0].message.content or '') + tool_info

ModelResponse._repr_markdown_ = _repr_markdown_



In [ ]:
#| export
class Chat:
    def __init__(self, model, tools=None, sp=None):
        self.model = model
        self.tools = tools and list(map(mk_tool_def, tools))
        self.msgs = [mk_msg(sp, 'system')] if sp else []
    
    def __call__(self, ct):
        self.msgs.append(mk_msg(ct))
        while True:
            res = completion(model=self.model, messages=self.msgs, tools=self.tools)
            self.msgs.append(res.choices[0].message)
            if not (tcs := res['choices'][0]['message']['tool_calls']): break
            for tc in tcs: self.msgs.append(ex_tool(tc))
        return res

In [ ]:
#| export
def add_numbers(
    a: int,  # First number to add
    b: int   # Second number to add  
) -> int:
    "Add two numbers together"
    return a + b

model = "groq/openai/gpt-oss-20b"
c = Chat(model, tools=[add_numbers])
c("What is 3 + 7 + 11 - 3? Use tools")

The result is **18**.

In [ ]:
c.msgs

[{'role': 'user', 'content': 'What is 3 + 7 + 11 - 3? Use tools'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"a":3,"b":7}', 'name': 'add_numbers'}, 'id': 'fc_f6fa8f86-2075-4fd6-8858-ed206f61fe37', 'type': 'function'}], function_call=None, provider_specific_fields=None, reasoning="User wants to compute expression using tool. We can call add_numbers for addition, but we need subtraction. We can do addition for 3+7=10, 10+11=21, then subtract 3: 21-3=18. But tool only adds. We could add 3+7, add result to 11, then subtract 3. But to subtract 3 we could add negative -3. So we can call add_numbers with a=3+7+11, b=-3. So we need intermediate: compute 3+7+11 using add_numbers twice? Let's do first a=3, b=7 => 10. Then add 10 and 11 => 21. Then add 21 and -3 => 18. We'll use tool."),
 {'role': 'tool',
  'content': '10',
  'tool_call_id': 'fc_f6fa8f86-2075-4fd6-8858-ed206f61fe37'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'argum

I want to create a python file from those messages. How do I do that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You have a couple of options:

1. **Mark messages as exported**: You can set `is_exported=True` on each code message you want to include, and solveit will export them to a module automatically.

2. **Quick manual approach**: I can help you extract the code from those messages and write it to a file. Want me to do that for you? Just let me know what filename you'd like (e.g., `agent_lib.py`).

Which approach would you prefer?

Ok I marked messages, but how do I create a script from them?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Once messages are marked as exported, you can create a module file by clicking the **"Export"** button in the toolbar (it looks like a download/export icon).

This will generate a `.py` file containing all the code from messages marked with `is_exported=True`, in order.

Did you find the Export button, or would you like me to show you how to do it programmatically instead?

programmatically

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>find_msgs(msg_type=code, re_pattern=., include_output=False)</summary>

```json
{
  "id": "toolu_01MNHbyafcTz4VTaACDr2s3Z",
  "call": {
    "function": "find_msgs",
    "arguments": {
      "msg_type": "code",
      "re_pattern": ".",
      "include_output": "False"
    }
  },
  "result": "[{'id': '_294072af', 'time_run': '2026-01-13T03:45:24.703531+00:00', 'is_exported': 1, 'skipped': 0, 'hidden': False, 'bookmark': {}, 'i_collapsed': 0, 'o_collapsed': 0, 'heading_collapsed': False, 'i_clamp': False, 'o_clamp': False, 'pinned': 0, 'content': 'import inspect, re, json\\nfrom litellm import completion, ModelResponse', 'msg_type': 'code'}, {'id': '_8491008d', 'time_run': '2026-01-13T03:45:26.180016+00:00', 'is_exported': 1, 'skipped': 0, 'hidden': False, 'bookmark': {}, 'i_collapsed': 0, 'o_collapsed': 0, 'heading_collapsed': False, 'i_clamp': False, 'o_clamp': False, 'pinned': 0, <TRUNCATED>"
}
```

</details>

I found all your exported messages. Here's a code snippet to create the file:

```python
from pathlib import Path

# Get exported code messages and join their content
exported = [m for m in find_msgs(msg_type='code') if m['is_exported']]
code = '\n\n'.join(m['content'] for m in exported)

Path('agent_lib.py').write_text(code)
print(f"Created agent_lib.py with {len(exported)} code blocks")
```

This will create `agent_lib.py` with all your exported code messages concatenated together. Want to use a different filename?